##AIML Module 1 - Lab 2
# Machine learning terms and metrics


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [4]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-4-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [5]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [6]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [7]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [8]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [9]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [10]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [11]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [12]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [13]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [14]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

In [15]:
'''
How is the accuracy of the validation set affected if we
increase the percentage of validation set? What happens when we reduce it?
'''

#-----> Here we are increasing the Validation set!

#Here we are using Validation data size of 25%
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
trainpred = NN(traindata, trainlabel, traindata)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is (for 75% val data):", trainAccuracy)
print("Validation accuracy of nearest neighbour is (for 25% val data):", valAccuracy)

print("-"*100)

#Here We are using validation data size of 50 %
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 50/100)
trainpred = NN(traindata, trainlabel, traindata)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is (for 50% data):", trainAccuracy)
print("Validation accuracy of nearest neighbour is (for 50 % val data):", valAccuracy)

print("-"*100)

#Here we are using validation data size of 75%
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 25/100)
trainpred = NN(traindata, trainlabel, traindata)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is (for 25% data):", trainAccuracy)
print("Validation accuracy of nearest neighbour is (for 75 % val data):", valAccuracy)

print("-"*100)

observation=""" We Could see that, when ever the validation data size is increased and train data size is decreased
the validation accuracy will be decreasing and train accuracy will is remained constant, hence that means model is
getting overfitted towards the train data. Because of that, prediction with validation data will make less
accuracy predictions
"""
print("Observation 1: ", observation)

print("="*100)
#-----> Here we are decreasing the Validation set!

#Here we are using Validation data size of 25%
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
trainpred = NN(traindata, trainlabel, traindata)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is (for 75% val data):", trainAccuracy)
print("Validation accuracy of nearest neighbour is (for 25% val data):", valAccuracy)

print("-"*100)

#Here We are using validation data size of 10 %
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 90/100)
trainpred = NN(traindata, trainlabel, traindata)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is (for 90% data):", trainAccuracy)
print("Validation accuracy of nearest neighbour is (for 10 % val data):", valAccuracy)

print("-"*100)

#Here we are using validation data size of 5%
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 95/100)
trainpred = NN(traindata, trainlabel, traindata)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is (for 95% data):", trainAccuracy)
print("Validation accuracy of nearest neighbour is (for 5 % val data):", valAccuracy)

print("-"*100)

observation=""" We can see that when the model is much exposed to the train data, it is getting better in prediction with the validation data,
Hence the accuracy with the validation data is increased.
"""

print("Observation 2:",observation)

Train accuracy using nearest neighbour is (for 75% val data): 1.0
Validation accuracy of nearest neighbour is (for 25% val data): 0.33925097276264593
----------------------------------------------------------------------------------------------------
Train accuracy using nearest neighbour is (for 50% data): 1.0
Validation accuracy of nearest neighbour is (for 50 % val data): 0.32343314341132257
----------------------------------------------------------------------------------------------------
Train accuracy using nearest neighbour is (for 25% data): 1.0
Validation accuracy of nearest neighbour is (for 75 % val data): 0.31270200387847447
----------------------------------------------------------------------------------------------------
Observation 1:   We Could see that, when ever the validation data size is increased and train data size is decreased
the validation accuracy will be decreasing and train accuracy will is remained constant, hence that means model is 
getting overfitted t

In [16]:
'''
How does the size of the train and validation set affect how well
we can predict the accuracy on the test set using the validation set?
'''
f"""When the model is trained more with training data, the model is much exposed to the crutial points of the data That is very useful in learning the non linear behaviour of the data. Hence, model will do great with the validation data."""

'When the model is trained more with training data, the model is much exposed to the crutial points of the data That is very useful in learning the non linear behaviour of the data. Hence, model will do great with the validation data.'

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [18]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [28]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.34462482100342917
test accuracy is  0.3666425295679459


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


In [26]:
f'''
Does averaging the validation accuracy across multiple splits give more consistent results?
'''
def Acc(alldata, alllabel, splitpercent, iterations, classifier=NN):
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    print(f'Iteration {ii+1}: {Accuracy(vallabel, valpred)}')

Acc(alltraindata, alltrainlabel, 75/100, 10, classifier=NN)

print("-"*100)

f'''By Averaging the validation accuracy across multiple splits give more consistent results, that is beacause by randomly spliting the data there may be some cases where our model is not correctly exposed to few data samples which are of different kind, Hence by cross validation this can be covered'''

Iteration 1: 0.34244256348246677
Iteration 2: 0.34404119666503186
Iteration 3: 0.33153108871958714
Iteration 4: 0.34649122807017546
Iteration 5: 0.3368089233753637
Iteration 6: 0.32820389516710746
Iteration 7: 0.3382422802850356
Iteration 8: 0.34207361077700266
Iteration 9: 0.3390748031496063
Iteration 10: 0.3480544282645023
----------------------------------------------------------------------------------------------------


'\nBy Averaging the validation accuracy across multiple splits give more consistent results, that is beacause by randomly spliting the data there may be some cases where our model is not correctly exposed to few data samples which are of different kind, Hence by cross validation this can be covered\n'

In [27]:
'''
Does it give more accurate estimate of test accuracy?
'''
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

f'''Yes this will give best estimate of test accuracy! '''

test accuracy is  0.34917953667953666


'Yes this will give best estimate of test accuracy! '

In [29]:
'''
What is the effect of the number of iterations on the estimate?
Do we get a better estimate with higher iterations?
'''
f''' The no.of iterations is completely arbitary, the model will get definetly better with high no.of iterations, but choosing the no of iterations will be dependent on size of data, if size is more then the change in the accuracies will can be determined! if dataset size is less, the accuracy after few interations will be nearly equal and can not able to get much difference to run more iterations!'''

' The no.of iterations is completely arbitary, the model will get definetly better with high no.of iterations, but choosing the no of iterations will be dependent on size of data, if size is more then the change in the accuracies will can be determined! if dataset size is less, the accuracy after few interations will be nearly equal and can not able to get much difference to run more iterations!'

In [30]:
'''
Consider the results you got for the previous questions. Can we deal with a very small
train dataset or validation dataset by increasing the iterations?
'''
f'''Increasing the number of iterations can help mitigate some of the issues associated with having very small training or validation datasets, but it may not completely solve the problem'''

'Increasing the number of iterations can help mitigate some of the issues associated with having very small training or validation datasets, but it may not completely solve the problem'